This code responsibility it to provide short review of the movie provided by the user through movie url

In [2]:
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import requests
from IPython.display import Markdown, display
from openai import OpenAI

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:2_000]

In [4]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [5]:
system_prompt="""
you are a funny movie review assistant. Provide a short review of the movie based on the plot, characters, and overall experience.
"""

user_prompt="""
Here is the website content.Please provide a review of the movie based on the plot, characters, and overall experience.
"""

In [6]:
def get_movie_review(url):
    """
    Get a movie review from the given url
    """
    contents = fetch_website_contents(url)
    messages = [
       {"role": "system", "content": system_prompt},
       {"role": "user", "content": user_prompt + contents} 
    ]

    openai = OpenAI()
    
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    return response.choices[0].message.content.strip()

In [7]:
get_movie_review("https://en.wikipedia.org/wiki/Manikarnika:_The_Queen_of_Jhansi")

'**Review: Manikarnika: The Queen of Jhansi**\n\nAre you ready for a historical drama where the real MVP is a queen who could probably out-warrior a dozen Vikings while simultaneously knitting a cardigan? Look no further than *Manikarnika: The Queen of Jhansi*! \n\nKangana Ranaut struts onto the screen as Rani Lakshmibai, and let me tell you, she’s not just fighting for her kingdom; she’s also fighting against the tyranny of budget constraints. The plot is as grand as it gets, weaving a tale of heroism against the British Raj with the subtlety of a cannonball to the chest. And who doesn’t love a film that kicks off with a sword fight and a dramatic narrative from Amitabh Bachchan? \n\nThe characters range from the undeniably fierce Rani Lakshmibai to a cast of supporting characters whose motivations occasionally get lost in the melee—kind of like losing your left sock in the dryer. Yet, every character adds a dash of flavor to the chaos, like too much salt on popcorn: it’s not quite ba

In [8]:
def review_movie(url):
    review_movie= get_movie_review(url)
    display(Markdown(review_movie))

In [9]:
review_movie("https://en.wikipedia.org/wiki/Manikarnika:_The_Queen_of_Jhansi")

**Movie Review: Manikarnika: The Queen of Jhansi**

If you ever wanted to see a royal makeover of the action genre, “Manikarnika: The Queen of Jhansi” is here to provide just that, sprinkled with a dash of historical drama and a pinch of women power! 

**Plot:** The film chronicles the life of Rani Lakshmibai — a fierce monarch who fought against British colonial rule in the 1850s. It’s like “Braveheart” but with a lot more saris and a lot less Mel Gibson yelling, "Freedom!" The plot ambitiously attempts to juggle valor, heartbreak, and heroic sacrifice, all while our heroine manages to ride horses, dodge arrows, and somehow still find time to look flawless.

**Characters:** Kangana Ranaut shines as the titular queen, embodying strength and charisma, giving such a powerful performance that it might just inspire you to don your own crown and lead a rebellion. The supporting cast, featuring notable actors like Jisshu Sengupta and Ankita Lokhande, add flavor, though sometimes it feels like they’re just there to nod in support while Kangana steals the show—like that one friend who always shows up looking fabulous no matter the occasion.

**Overall Experience:** The visuals are a feast for the eyes, with grand battle scenes that make you feel like the world might just end if you blink. The music complements the intensity, though sometimes it feels like it’s trying a bit too hard to evoke emotions, like that overly enthusiastic friend at a party who just can’t stop dancing. 

In summary, while “Manikarnika” may falter on some historical accuracy and pacing, it delivers an exhilarating experience that celebrates bravery and resilience. Just make sure to keep some popcorn handy, because there’s enough drama here to keep you munching for the full 150 minutes! Great for history buffs, fans of epic storytelling, or anyone in need of a good dose of empowerment, just don’t expect it to go easy on the 'drama meter'!